### Problema 9: maniobra combinada GEO (Ex. Sep. 2010)
Se está estudiando la viabilidad de emplear un cierto lanzador para poner en órbita un satélite
geoestacionario (r_GEO = 42.164 km). Dicho lanzador es capaz de situar dicho satélite en una órbita
circular de 300 km de altitud y con una inclinación de 10º.
Para estudiar las implicaciones de la elección de este lanzador en el subsistema propulsivo se va
a evaluar:

- El presupuesto total de DV
- El DV que tiene que ser capaz de proveer el sistema propulsivo en un solo impulso
- El tiempo requerido para realizar cada maniobra, teniendo en cuenta que no se harán
maniobras en órbitas sucesivas; se debe dejar una órbita completa sin maniobras antes
de realizar la siguiente maniobra

empleando varias estrategias de maniobras distintas.

Para ello calcule con 4 cifras significativas cada uno de los valores que se indican en la
siguiente tabla:

    - Cambio de plano a 300 km, seguido de una transferencia a GEO
    - Transferencia de Hohmann a altitud de GEO, seguido de un cambio de plano
    - Maniobra combinada (plano y altitud) con todo el cambio de plano en el perigeo
    - Maniobra combinada (plano y altitud) con todo el cambio de plano en el apogeo
    - Maniobra combinada (plano y altitud) con todo el cambio de plano en el perigeo de 0,917º y en el apogeo de 9,083º

Valores:

    - DV total (km/s),
    - DV_max en un único impulso (km/s),
    - t requerido (h)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mu = 0.3986e6   # km^3 s^-2
rE = 6.378e3    # km
# inicial and final conditions:
ri = rE + 300   # km
rf = 42_164     # km
Vi = np.sqrt(mu/ri) # km/s
Vf = np.sqrt(mu/rf) # km/s

# inclinations (rad)
i_i = 10 * np.pi/180
i_f = 0
Deltai = i_f - i_i
# semi-major axis of trasnfer orbit
at = (ri + rf)/2

Vta = np.sqrt(2*mu/ri - mu/at)
Vtb = np.sqrt(2*mu/rf - mu/at)

TABLE = np.zeros((5,3))

### CASE 1:
`Cambio de plano a 300 km, seguido de una transferencia a GEO`

Cambias de plano en LEO. Esperas una vuelta en la circular. Das impulso de hohmann y al llegar al apogeo, recirculas.


In [3]:
# POINT A:
# first impulse: change of plain:
DeltaV_a_1 = 2*Vi*np.sin(Deltai/2)
# second impulse: transfer orbit:
DeltaV_a_2 = Vta - Vi
# POINT B:
DeltaV_b = Vf - Vtb

# TOTAL IMPULSE:
DeltaV_tot = abs(DeltaV_a_1) + abs(DeltaV_a_2) + abs(DeltaV_b)
# MAX SINGLE IMPULSE:
DeltaV_max = max(DeltaV_a_1, DeltaV_a_2, DeltaV_b)

# TIME:
t_cir = 2*np.pi * np.sqrt(ri**3 / mu)  # one complete LEO orbit
t_trans = np.pi * np.sqrt(at**3 / mu)  # half transfer orbit
time_hours = (t_cir + t_trans)/3600

print(f'DeltaV total (km/s) = {DeltaV_tot:.4f} \nDeltaV_max (km/s) = {DeltaV_max:.4f} \nttime (h) = {time_hours:.4f}')
TABLE[0,:] = [DeltaV_tot, DeltaV_max, time_hours]

DeltaV total (km/s) = 5.2393 
DeltaV_max (km/s) = 2.4258 
ttime (h) = 6.7836


### CASE 2:
`Transferencia de Hohmann a altitud de GEO, seguido de un cambio de plano`

Entendiéndolo como: impulso para cambio de altura, te esperas otra vuelta en la elíptica, recirculas en el apogeo. Luego, te esperas una vuelta en la GEO girada 10º y luego haces un tercer impulso para el cambio de plano.


In [6]:
# POINT A:
# impulse: transfer orbit:
DeltaV_a = Vta - Vi
# POINT B:
# impulse: recircularization:
DeltaV_b_1 = Vf - Vtb
# impulse: change of plain:
DeltaV_b_2 = 2*Vf*np.sin(Deltai/2)

# TOTAL IMPULSE:
DeltaV_tot = abs(DeltaV_a) + abs(DeltaV_b_1) + abs(DeltaV_b_2)
# MAX SINGLE IMPULSE:
DeltaV_max = max(DeltaV_a, DeltaV_b_2, DeltaV_b_2)

# Time:
t_trans = 3/2 * 2*np.pi * np.sqrt(at**3 / mu)   # one and a half transfer orbits
t_GEO = 2*np.pi * np.sqrt(rf**3 / mu)           # one complete GEO orbit
time_hours = (t_trans + t_GEO)/3600
print(f'DeltaV total (km/s) = {DeltaV_tot:.4f} \nDeltaV_max (km/s) = {DeltaV_max:.4f} \nt (h) = {time_hours:.4f}')
TABLE[1,:] = [DeltaV_tot, DeltaV_max, time_hours]

DeltaV total (km/s) = 4.4286 
DeltaV_max (km/s) = 2.4258 
t (h) = 39.7594


In [5]:
Vf

3.0746645801808263

### CASE 3
`Maniobra combinada (plano y altitud) con todo el cambio de plano en el perigeo`

Combinada, todo el cambio de plano en el perigeo. Pero al llegar al apogeo de la de transferencia, te tienes que esperar otra vuelta entera.

In [7]:
# POINT A:
# combined impulse
Phi_a = Deltai
DeltaV_a = np.sqrt(Vi**2 + Vta**2 - 2*Vi*Vta*np.cos(Phi_a))
"""In circular orbits and combined maneouvres, \phi is equal to \Deltai"""

# POINT B:
DeltaV_b = Vf - Vtb

# TOTAL IMPULSE:
DeltaV_tot = abs(DeltaV_a) + abs(DeltaV_b)
# MAX SINGLE IMPULSE:
DeltaV_max = max(DeltaV_a, DeltaV_b)

# Time:
t_ho = 3/2 * 2*np.pi * np.sqrt(at**3 / mu)  # one complete transfer orbit and half more
time_hours = t_ho/3600
print(f'DeltaV total (km/s) = {DeltaV_tot:.4f} \nDeltaV_max (km/s) = {DeltaV_max:.4f} \nt (h) = {time_hours:.4f}')
TABLE[2,:] = [DeltaV_tot, DeltaV_max, time_hours]

DeltaV total (km/s) = 4.3421 
DeltaV_max (km/s) = 2.8753 
t (h) = 15.8251


### CASE 4:
`Maniobra combinada (plano y altitud) con todo el cambio de plano en el apogeo`

Combinada con todo el cambio de plano en el apogeo. Lo mismo, esperándote una vuelta más en la de trasnferencia.

In [8]:
# POINT A:
# hohmann impulse
DeltaV_a = Vta - Vi
# POINT B:
# combined impulse:
Phi_b = Deltai
DeltaV_b = np.sqrt(Vtb**2 + Vf**2 - 2*Vtb*Vf*np.cos(Phi_b))

# TOTAL IMPULSE:
DeltaV_tot = abs(DeltaV_a) + abs(DeltaV_b)
# MAX SINGLE IMPULSE:
DeltaV_max = max(DeltaV_a, DeltaV_b)

# Time:
t_ho = 3/2 * 2*np.pi * np.sqrt(at**3 / mu)  # one complete transfer orbit and half more
time_hours = t_ho/3600
print(f'DeltaV total (km/s) = {DeltaV_tot:.4f} \nDeltaV_max (km/s) = {DeltaV_max:.4f} \nt (h) = {time_hours:.4f}')
TABLE[3,:] = [DeltaV_tot, DeltaV_max, time_hours]

DeltaV total (km/s) = 3.9429 
DeltaV_max (km/s) = 2.4258 
t (h) = 15.8251


### CASE 5:
`Maniobra combinada (plano y altitud) con todo el cambio de plano en el perigeo de 0,917º y en el apogeo de 9,083º`

"Maniobra combinada (plano y altitud) con el cambio de plano en el perigeo de 0,917º y en el apogeo de 9,083º."

In [9]:
# POINT A:
# combined impulse:
Phi_a = 0.917 * np.pi/180
DeltaV_a = np.sqrt(Vi**2 + Vta**2 - 2*Vi*Vta*np.cos(Phi_a))
# combined impulse:
Phi_b = 9.083 * np.pi/180
DeltaV_b = np.sqrt(Vtb**2 + Vf**2 - 2*Vtb*Vf*np.cos(Phi_b))

# TOTAL IMPULSE:
DeltaV_tot = abs(DeltaV_a) + abs(DeltaV_b)
# MAX SINGLE IMPULSE:
DeltaV_max = max(DeltaV_a, DeltaV_b)

# Time:
t_ho = 3/2 * 2*np.pi * np.sqrt(at**3 / mu)  # one complete transfer orbit and half more
time_hours = t_ho/3600
print(f'DeltaV total (km/s) = {DeltaV_tot:.4f} \nDeltaV_max (km/s) = {DeltaV_max:.4f} \nt (h) = {time_hours:.4f}')
TABLE[4,:] = [DeltaV_tot, DeltaV_max, time_hours]

DeltaV total (km/s) = 3.9384 
DeltaV_max (km/s) = 2.4299 
t (h) = 15.8251


In [12]:
import pandas as pd
df = pd.DataFrame(TABLE)#, col={'DeltaV', 'DeltaV', 'DeltaV'})
